# Summary
This interactive notebook has the objective of demonstrating the relationships between canopy structure, turbulent transport of heat and water and surface temperature.

To this purpose we will use simulations based on energy combination models, both the "big  leaf" [Penman-Monteith](https://www.sciencedirect.com/book/9780123869104/principles-of-environmental-physics "https://www.sciencedirect.com/book/9780123869104/principles-of-environmental-physics") model as well the dual source [Shuttleworth-Wallace](https://rmets.onlinelibrary.wiley.com/doi/10.1002/qj.49711146910 "https://rmets.onlinelibrary.wiley.com/doi/10.1002/qj.49711146910") model.

> The most widely application of Penman-Monteith is the calculation of referece ET, e.g. [FAO's reference ET](https://www.fao.org/3/x0490e/x0490e06.htm#chapter%202%20%20%20fao%20penman%20monteith%20equation "https://www.fao.org/3/x0490e/x0490e06.htm#chapter%202%20%20%20fao%20penman%20monteith%20equation"), which estimates the evapotranspiration for a well watered crop used as reference. This reference crop is defined as a perfectly homogeneous grass layer clipped at 12cm height.

# Instructions
Read carefully all the text and follow its instructions.

Once each section is read, run the jupyter code cell underneach (marked as `In []`) by cliking the icon `Run`, or pressing the keys ALT+INTRO of your keyboard. A graphical interface will then be displayed which you could interact with and performe the assigned tasks.

As a starters, please run the following cell in order to import all the required libraries in this notebook. Once the cell below is run an aknowledgement message should be printed on screen.

In [ ]:
from ipywidgets import interactive, fixed
from IPython.display import display
import functions.tseb_and_resistances as fn

# Basic equations
As a remainder, the estimation of evapotranspiration (ET), or latent heat flux ($\lambda E$) in terms of energy, can be obtained from the energy balance equation:

$$\lambda E \approx R_n - G - H$$

In the previous exercise we have already seen the theory and processes related to net radiation ($R_n$) and soil heat flux ($G$). Now we will focus in the sensible heat flux ($H$) to finally get $\lambda E$ or ET.

The effective transport of sensible heat can be computed as:

$$H = \rho_{air} C_p \frac{T_0 - T_a}{r_a}$$

* [$\rho_{air}$](https://github.com/hectornieto/pyTSEB/blob/9d1f02ec2968bb323c07528ba44144c3733f3737/pyTSEB/meteo_utils.py#L151 "https://github.com/hectornieto/pyTSEB/blob/9d1f02ec2968bb323c07528ba44144c3733f3737/pyTSEB/meteo_utils.py#L151") and [$C_p$](https://github.com/hectornieto/pyTSEB/blob/9d1f02ec2968bb323c07528ba44144c3733f3737/pyTSEB/meteo_utils.py#L65 "https://github.com/hectornieto/pyTSEB/blob/9d1f02ec2968bb323c07528ba44144c3733f3737/pyTSEB/meteo_utils.py#L65") are respectively the density of air and its specific heat, and they can be assumed practically constant.
* $T_0$ (K) is the aerodynamic temperature.
> **DO NOT CONFUSE IT WITH THE RADIOMETRIC SURFACE TEMPERATURE THAT IS ESTIMATED WITH THERMAL INFRARED REMOTE SENSING**
* $T_a$ (K) is the air temperature measured at a height above ground $z_T$, and sensitively above the canopy height ($h_c$)
* $r_a$ is the aerodynamic resistance to heat transport (s/m), which reciprocal would be the aerodynamice conductance.
> **DO NOT CONFUSE THIS AERODYNAMIC CONDUCTANCE WITH THE LEAF STOMATA CONDUCTANCE**

This fundamental equation can be interpreted in a way that the heat exchange between the land surface and the atmosphere ($H$) is driven by a gradient of temperatures between the surface and the nearest layers of air above the canopy. This transport or exchange of heat is modulated by the aerodynamic resistance, which includes a series of complex mechanical and convective processes that enhances or impedes the transport of heat between the surface and the air:

$$r_{a}=\frac{\left[\ln\left(\frac{z_{T}-d}{z_{0H}}\right)+\Psi_{H}\right] \left[\ln\left(\frac{z_{U}-d}{z_{0M}}\right)+\Psi'_{M}\right]}{k^2\,U}$$

* $U$ is the wind speed (m/s) measured at a height above ground $z_U$, and sensitively above the canopy height ($h_c$).
* $z_{T}$ is the height (m) at which air temperature is measured or modelled.
* $z_{U}$ is the height (m) at which wind speed is measured or modelled.
* $d$ and $z_{0M}$ are respectively the zero-plane displacement height and surface roughness length for momentum transport. Usually these two variables can be estimated using simplified formulae based on canopy/obstacle heigth ($h_c$)

$$z_{0M} = 0.125 h_c\\
d_{0} = 2/3 h_c$$

* $z_{0H}$ is the roughness length for heat transport. Likewise, its estimation is usually computed as a fraction of  $z_{0M}$ (e.g. $$z_{0H} = 0.1 z_{0M}$$ for dense vegetation)

* $k=0.41$ is von Kàrman constant.

* $\Psi'_{H}$ and $\Psi'_{M}$ are semi-empirical functions that modulate the aerodynamic resistance and the vertical wind attenuation as function of atmospheric stability.
> If you want to know more details on their calculation, you can check the Python code for [$\Psi'_{H}$](https://github.com/hectornieto/pyTSEB/blob/600664efd3e5ac4edab84e84fa5cb9d55c58c46f/pyTSEB/MO_similarity.py#L179 "https://github.com/hectornieto/pyTSEB/blob/600664efd3e5ac4edab84e84fa5cb9d55c58c46f/pyTSEB/MO_similarity.py#L179")  and for [$\Psi'_{M}$](https://github.com/hectornieto/pyTSEB/blob/600664efd3e5ac4edab84e84fa5cb9d55c58c46f/pyTSEB/MO_similarity.py#L244 "https://github.com/hectornieto/pyTSEB/blob/600664efd3e5ac4edab84e84fa5cb9d55c58c46f/pyTSEB/MO_similarity.py#L244").

From the the equation above we can arithmetically deduce that the aerodynamic resistance to heat and vapour transport decreases with wind speed and with the surface aerodynamic roughness.

In the following taks we are going to try disentangling step by step all those processes that would allow us better undertand and estimate $r_a$.

# The wind profile
The wind speed profile with respect to the height above ground can be estimated according to a logarithmic profile. Ignoring in a first step the effects of convective turbulence, and thus considering a neutral atmospheric stability:

$$u\left(z\right) =\frac{u_*}{k}\left[\log\left(\frac{z - d}{z_{0M}}\right)\right]$$

> This logarithmic wind attenuation is closely related to the aerodynamic resistance. You can check the Python code [here](https://github.com/hectornieto/pyTSEB/blob/9d1f02ec2968bb323c07528ba44144c3733f3737/pyTSEB/wind_profile.py#L38 "https://github.com/hectornieto/pyTSEB/blob/9d1f02ec2968bb323c07528ba44144c3733f3737/pyTSEB/wind_profile.py#L38").

The wind attenuation will be stronger at larger surface roughness, expressed by the variables $z_{0M}$ and $d_{0}$. 

$u_{*}$ is the friction velocity. Likewise, simplifying and neglecting the effects of atmospheric stability, $u_{*}$ is computed as:

$$u_* = \frac{k\,U} {\log\left(\frac{z_u - d_0}{z_{0M}}\right))}$$

> You can see the Python code [here](https://github.com/hectornieto/pyTSEB/blob/600664efd3e5ac4edab84e84fa5cb9d55c58c46f/pyTSEB/MO_similarity.py#L357 "https://github.com/hectornieto/pyTSEB/blob/600664efd3e5ac4edab84e84fa5cb9d55c58c46f/pyTSEB/MO_similarity.py#L357").

Therefore, based on measurement of wind speed at a given height we could extrapolate the wind profile to estimate with wind speed right above the canopy ($h_c$):

$$u_C =\frac{u_*}{k}\left[\log\left(\frac{h_c - d}{z_{0M}}\right)\right]$$

## The wind profile within the canopy
As the wind circulates through the canopy, it is also attenuated mainly by the presence of foliage. This attenuation within the canopy and towards the ground can have implications on the heat and vapour exchange between the canopy and the ground.

Usually this attenuation is formulated assuming that the canopy is horizontally and vertically homogeneous. However most canopies, and in particular woody vegetation, show a foliar density that varies with height: plants usually show a stem from with branches diverge, starting at a certain height above ground, showing canopies with a shape more or less irregular.

In the following interactive plot we are going to simulate the wind attenuation above and below the canopy, and how this profile diverges according to the canopy structure. We are going to compare a profile for both a vertically heterogeneous and heterogeneous canopy. In addition, we are going to see the standard wind profile applied in the reference ET method proposed by FAO. For this task we are keeping a constant wind speed ($U=5$ m/s) measured 10m above ground ($z_U=10$ m), and assuming a neutral atmospheric stability.

In order to characterize our canopy we are going to need:

* Leaf Area Index ($LAI$) and canopy height ($h_c$), these two parameters are common for the homogeneous and heterogeneous canopy.

* The height above which the foliage starts ($h_{bottom}$), expressed as a ratio to the canopy height
> By default this value is set at 0.5m, meaning that the foliage starts at a height of $0.5 h_c$ m. 

* The height at with the maximum foliage density is located ($h_{max}$), expressed as a ratio of the canopy length.

The plot on the left hand side in the graph shows the foliage relative density of the simulated canopy. 

The plot on the right hand side shows the wind speed attenuation from 10m to the ground (0m) for both the vertical heterogeneous (solid blue line) and homogeneous cases (dashed blue line). A blue star indicates the wind speed right above the canopy, and the dashed black line shows the standard profile for the reference ET crop in FAO method (a homogeneous grass layear with $h_c=0.12$ m and $LAI\approx3$).

In [ ]:
w_wind = interactive(fn.wind_profile_heterogeneous, lai=fn.w_lai, zol=fixed(0.), h_c=fn.w_hc, 
                     hb_ratio=fn.w_hb_ratio, h_c_max=fn.w_h_c_max)
display(w_wind)

* See how different the wind profile can result compared to the profile for the FAO reference crop. This is particular evident when $h_c$ is significantly larger than 0.12 m. In addition with higher canopies you will see more clearly the differences between a profile for a vertically homogenous crop and the heterogenous one.

* As a rule of thum, the stronger the attenuation is, the more turbulent the exchange of heat and vapour will be. This turbulence will enhance more eddies and a more efficient transport of heat an vapour between the soil, the plant and the atmosphere, since eddies removes towards the atmosphere the parcels of (warm and humid) air in contact with the land surface.

> [This video shows a simulation of eddies while surface roughness in increased. In this case it corresponds to an airfoil](https://youtu.be/5YwnY0wPphA "https://youtu.be/5YwnY0wPphA").

* See also that with higher leaf density (expressed in terms of `LAI`) the wind attenuation within the canopy will be stronger. This would result in distinct heat and water exchange between the canopy and the ground.

* Finally see that the stronger attenuation within the canopy occurs around the height of maximum foliage density. This is due to the fact that the wind is encountering a larger amount of obstacles (leaves) and thus looses kinetic energy and hence speed. On the contrary, areas closer to the ground, where the foliage density is minimal the wind is less attenuated.

## Effect in the aerodynamic resistance

In this task you will explore how the aerodynamic resistance varies with surface roughness (equivalent to canopy height), wind speed and atmospheric stability. 

### The atmospheric stability and convective turbulence
So far we saw that arithmetically the aerodynamic resistance to heat and vapour transport decreases with wind speed and roughness. In other words the exchange of heat and vapour between the surface and the atmosphere becomes more efficient with stronger winds and rougher surfaces.

We are going to evaluate now that with more unstable atmospheres due to convection the turbulence is enhanced and thus the heat exchange is as well more efficient. 

The atmoshperic stability can be expressed in relative terms as a ratio between aerodynamic roughness length ($z_{0m}$) and the Monin-Obukhov length ($\xi=z_{0m}/L$), which the latter depends as well on the sensible heat flux ($H$).

> If you want more details on the calculation of the Monin-Obukhov length click [here](https://github.com/hectornieto/pyTSEB/blob/600664efd3e5ac4edab84e84fa5cb9d55c58c46f/pyTSEB/MO_similarity.py#L134 "https://github.com/hectornieto/pyTSEB/blob/600664efd3e5ac4edab84e84fa5cb9d55c58c46f/pyTSEB/MO_similarity.py#L134")

The more negative this $\xi$ coefficient is, the more unstable the atmosphere is and thus more efficient the transport of heat and vapour would be due to the convection of warm air upwards the cooler atmosphere.

> As a remainder, the density of warm air is smaller than cool air, provoking a vertical (and turbulent) transport of air upwards. You can see a video of this phenomenon [here](https://youtu.be/OM0l2YPVMf8 "https://youtu.be/OM0l2YPVMf8").

> At daytime, the land surface is absorbing heat (mainly to to the sun irradiance) and therefore unstable conditions usually occur ($\xi < 0$). At nightime on the other hand, the land surface can be cooler than the air and thus thermal inversion phenomena and atmospheric stability can occur ($\xi > 0$). 

The next interactive plot will let you evaluate the joint effect of atmospheric stability, wind speed and canopy height/roughness in the aerodynamic resistance.

In [ ]:
w_ra = interactive(fn.plot_aerodynamic_resistance, zol=fn.w_zol, h_c=fn.w_hc)
display(w_ra)

> All parameters in all the plots are synchronized: any parameter change in one of the interactive plots is as well changed in the other plots, being all the graphs updated accordingly. This will allow a quick intercomparison between graphs and simulations.

* See how the aerodynamic resistance reduces exponentially with wind speed, but also with surface roughness/canopy height. This issue can have crucial implications in order to interpret the gradient of temperature between the surface and the air. 

> The same temperature gradient can indicate a water stress or not and the degree of it, depending on the surface roughnees and the actual meteorological conditions.

* Change now the stability coefficient and see how $r_a$ changes. The aerodynamic resistance is reduced with more negative $xi$ values for a given wind speed and surface roughness. On the opposite, with stable conditions ($xi$ > 0) the vertical transport of heat and vapour is reduced (larger values of $r_a$).

# The evapotranspiration and the sensible heat flux
In this second part of this exercise we are going to integrate everything we have seen so far to evaluate the effect on the evapotranspiraion and the land surface temperature. As mentioned before we will use two popular energy combination models to perform our simulations.

The main advantage of Penman-Monteith is its relative simplicity (this it is the model used to estimate FAO's reference ET), while its main disadvange is that is not fully able to partition fluxes between soil and canopy (e.g. separation of soil evaporation from canopy transpiration. This issue also has implications in modelling ET for sparse vegetation or row crops with this model.

![Penman-Monteith](./input/figures/penman_monteith_model.png "Resistances layout in the Penman-Monteith model")

On the other hand, the Shuttleworth-Wallace model is more adapted to this sparse vegetation conditions, and as well is able to partition fluxes between soil and canopy, allowing thus the separation between soil evaporation and canopy transpiration de la evaporación del suelo y la transpiración de la planta. However this model is more complex than Penman-Monteith and thus requires additional inputs.

![Shuttleworth-Wallace](./input/figures/shuttleworth_wallace_model.png "Resistances layout in the Shuttleworth-Wallace model")

As additional variables required by the Shuttleworth-Wallace model is the soil resistance of vapour transport  ($R_{ss}$) which is related to the topsoil moisture content.

> $R_{ss}$ values closer to 0 would indicate topsoil saturated or even flooded, whereas larger values will indicate a drier soil surface.


## The effect of stomatal conductance
The leaf stomatal conductance or its reciprocal the leaf stomatal resistance ($g_s = 1 / r_{st}$) in an indicator of crop water stress. $g_s$ usually show values around a 0.4-0.5 mmol/m²s¹ when the crop is well watered.

> According to FAO guidelines, the stomatal resitance of the reference well watered crop is 100 s/m, which is equivalent to a stomatal conductance of 0.415 mmol/m²s¹ (at standard atmospheric conditoins of 25ºC at sea level).

Once a soil water deficit at the root-zone appears, the typical phisological response of plants is to close partially or totally their stoma, with the aim of avoiding significant losses of water. This closure has the consequence of a lower CO$_2$ assimilation throught he stoma and therefore a readuction in the plants productivity. This phisological stomata closure is mathematically reflected in a decrease (increase) of stomata conductance (resistance).


## Micro-advection: flux interactin between ground and the canopy
Any exchange of heat and vapour between the ground, vegetation and the atmosphere is produced under a turbulent transport. Therefore if the ground surface is dry and receives enough radiation, this one will heat up and thus will enhance a tranport of dry and hot air towards the canopy. This would enhance the evaporative demand of the canopy (warming and drying the air around the leaves). Inversely, a dense and well irrigated canopy can provide cool and moist air towards the ground environment, decreasing its evaporative demand.

The following interactive plot will run simulations of Penman-Monteith and Shuttleworth-Wallace models for a range of LAIs, given the following standard meterological forcing:

* Solar irradiace at 300 W/m².
* Air temperature at 25ºC (298K).
* 50% relative humidity, which is equivalte to a vapour pressure deficit of $VPD\approx$ 15mb.
* Wind speed of 5 m/s measured at 10m above ground.

... and fixed surface albedo of 0.23 as defined for FAO's reference ET.

The upper pannel of the plot shows the simulated daily ET for the Shuttleworth-Wallace (blue line) and Penman-Monteith (red line) models. In addition it shows the FAO reference ET value with a black star.

The middle pannel shows the ET partitioning between soil evaporation (yellow line) and canopy transpiration (green line) as simulated by Shuttleworth-Wallace.

The lower pannel shows the Shuttleworth-Wallace simulated soil (yellow line), canopy (green) and aerodynamic temperatures (black line), expressed as the respective differeces from the air temperature.
<a id='ET_simulations'></a>

In [ ]:
w_et = interactive(fn.fluxes_and_resistances, r_ss=fn.w_r_ss, g_st=fn.w_g_st, h_c=fn.w_hc)
display(w_et)

* Observe how the graphs change with $R_{ss}$ (e.g. the topsoil moisture). See how the total ET in Shuttleworth-Wallace changes as opposed to the Penman-Monteith that can not directly consider the topsoil moisture. See how ET increases with LAI for the given parameters, since surface resistance decreases with LAI ($R_c = r_{st}/LAI$). 

> Remind that $R_{ss}$ represents the topsoil moisture condition. Indeed it can happen (more usual than one could expect) of having a soil surface dry ($R_{ss} >> 1000$ s/m) and yet having a well watered canopy ($g_{st} >> 0 mmol/m²s), since plants extract water deeper in the soil, where water is still stored.

* See in the middle pannel how a transpiration dominates the ET at larger values of LAI. In the previous exercise we showed that at larger LAI values the canopy will intercept and absorb more energy, and thus will have larger evaporative capacity.

* Look now to the ET values at $LAI\rightarrow0$ when $R_{ss}\rightarrow0$. Penman-Monteith, since it is a "big-leaf" model and does not directly incorporate the transport of water from the soil surface, is not able of modelling the soil surface evaporation. On the other side with $R_{ss}$ values above 5000 s/m both models yield very similar values of ET.

* Observe in the lower pannel how temperatures decrease with LAI: a dense canopy dissipates the heat more efficiently than a less dense canopy, even under the same water stress conditions. 

* Check also how soil temperature increases with $R_{ss}$. 

> You could even perceive how the leaf temperature slightly changes with $R_{ss}$, especially at lower LAI values soil. This is related to the flux interaction between ground and canopy that we mentioned before: a dry and warm (moist and cool) soil transports air towards the canopy, which can increase (decrease) the evaporative demand and thus decrease (increase) the canopy temperature. 

* Keep a value of $R_{ss}=5000$ fixed (dry soil surface) and check now how ET changes with decreasing values of stomatal conductance, and thus simulating more severe crop water stress conditions. The ET for both Penman-Monteith and Shuttleworth-Wallace decreases and deviates further from the FAO's reference ET value (optimal water conditions). In addition, the more stressed the plant is (i.e. at lower $g_{st}$ values) the lower the transpiration, with no transpiration for $g_s=0$ mmol/m²s¹. Finally watch that with lower $g_{st}$ the hotter the leaf and aerodynamic temperatures.

> You could even perceive how the soil temperature slightly changes with $g_{st}$, especially at larger LAI values soil. This is again related to the flux interaction between ground and canopy that we mentioned before: a dry and warm (moist and cool) canopy transports air towards the ground, which can increase (decrease) the evaporative demand and thus decrease (increase) the soil temperature. 

* See now the effect of canopy height/roughness. Restore the values of stomata conductance back to $g_s=0.41$ mmol/m²s¹ and $R_{ss}=5000$. Increase now the canopy height, from an initial value of 0.12m, and observe how ET and temperature changes with canopy roughness for a given moisture status. 

# About the surface temperature
We alredy mentioned that the aerodynamic temperature ($T_0$), which is the temperature that holds valid the sensible heat flux equation, is different to the radiometric surface temperature (LST) estimated with Earth Observation remote sensing. It is therefore to be aware of such difference and thus be able to correctly interpret LST when estimating sensible heat flux and/or ET.

> Indeed, the different remote sensing ET models mainly differ on how they use the LST in order to retrieve $H$ and/or $\lambda E$.

Firstly, since heat is exchanged less efficiently than momentum, it is needed to correctly define the aerodynamic roughness length for heat transport ($z_0H$), and inndeed the relationship $z_{0H} \approx 0.1 z_{0M}$ might not be valid for any type of surface.

Secondly is the fact that the radiometric surface temperature could significantly change depending on the view observation angle. This is particular relevant for Earth Obsevation sensors with a wide field of view or wide swath, such as Sentinel-3 SLSTR, with view zenith angles from nadir ($VZA = 0$º) up to VZAs of 50º or 60º. The reason of this issue is that the sensor might see a different proportion of (cool) vegetation and (warm) soil. Indeeed, we saw in the previous exercise that the canopy transmittance depends on the canopy density, the leaf angle distribution, and the incidence/illumination angle. This effect is perfectly reciprocal when we deal with de sensor geometry instead of the solar geometry.

The interactive plot below show this effect. You can define the sensor's view zenith angle as well as the dominant leaf inclination angle. According to the soil and canopy temperatures estimated by the [previous task](#ET_simulations), and after setting as well the soil and leaf emissivity, the graph will display the estimated LST for a range of LAIs.

> You can modify the simulation parameters [here](#ET_simulations) and the changes will be reflected in the simulated LST.

The plot shows two different methods to estimate the radiometric land surface temperature:
* A detailed calculation based on the 4SAIL model that include thermal emission, multiple thermal scattering between soil and vegetation.
> See the detailed code [here](https://github.com/hectornieto/pypro4sail/blob/f6825f5b1325259cb014dcb934ccff2a9442a3ae/pypro4sail/four_sail.py#L445 "https://github.com/hectornieto/pypro4sail/blob/f6825f5b1325259cb014dcb934ccff2a9442a3ae/pypro4sail/four_sail.py#L445")

* A simplified method considering a weigted average based on the fraction of vegeation observed by the sensor:
$$LST^4=f_c\left(\theta\right)T_{C}^4+\left[1-f_{c}\left(\theta\right)\right]T_{S}^4$$
> See the detailed code [here](https://github.com/hectornieto/pyTSEB/blob/9d1f02ec2968bb323c07528ba44144c3733f3737/pyTSEB/TSEB.py#L2644 "https://github.com/hectornieto/pyTSEB/blob/9d1f02ec2968bb323c07528ba44144c3733f3737/pyTSEB/TSEB.py#L2644")

In [ ]:
w_fveg = interactive(fn.get_land_surface_temperature,
                     vza=fn.w_vza, leaf_angle=fn.w_leaf_angle, temperatures=fixed(w_et), 
                     e_v=fn.w_ev, e_s=fn.w_es)
display(w_fveg)

* Notice that with increasing VZA the LST is decreasing, being this decrease more significant at wider VZAs (VZA > 35º) and for intermediate values of LAI.

* There might be significant differences between the aerodynamic temperature and the LST, in particular for less developed and sparse canopies (LAI < 2).  

# Application with real data 

To conclude this exercise we are going to apply everythig we have learn in this session. To this purpose we are going to use real data using in situ measurements of an olive orchard in southern Spain. 

The site is a traditional olive orchard (37.914998ºN, 3.227659ºW, 366 m elevation) with with a ground cover of ca. 20% and canopies of 4m  height. The meteorological forcing, proximal surface temperature and actual ET are found in [./input/meteo/meteo_olive.csv](./input/meteo/meteo_olive.csv), which data were extracted from the ICOS portal.

![ES-Cnd](./input/figures/ES-Cnd.png "Location of ES-Cnd flux site")

> For mor information on the site and get the whole timeseries click [here](https://meta.icos-cp.eu/objects/Sqh69qK2l4QpSBo3U5ZAqWFa "https://meta.icos-cp.eu/objects/Sqh69qK2l4QpSBo3U5ZAqWFa").

On the otyher hand, we have extracted timeseries of Earth Observation LAI data, extracted form the LAI/fAPAR VIIRS product at 300m spatial resolution. These data are found in [./input/canopy/LAI_FAPAR_ES-Cnd.csv](./input/canopy/LAI_FAPAR_ES-Cnd.csv).

The following interactive plot shows the timeseries for the relevant fluxes ($R_n$, $\lambda E$), hourly radiometric LST and daily LAI and ET.

In [ ]:
w_ts = interactive(fn.create_flux_timeseries,
                   df=fixed(fn.METEO_DATA), daily_df=fixed(fn.METEO_DAILY_DATA),
                   date_range=fn.w_dates, include_obs=fixed(False))
display(w_ts)

# ET retrieval with proximal LST using TSEB

Based on the dual source Shuttleworth-Wallace model we have been working, it is possilble to invert it in order to estimate $\lambda $E and ET using the radiometric LST as boundary condition. This is the basis of the Two-Source Energy Balance Model (TSEB). 

In this case we will use an estimated proximal LST, derived form the Eddy Covariance radiometers deployed in the site, together with meteorological forcing inputs (irradiance, wind speed, and air temperature). TSEB performs an iterative process in order to derive the component soil and leaf temperatures, an thus estimate soil and canopy sensible and latent heat fluxes. Besides, TSEB is able to incorporate all processes previously described, in an effient manner, in such a way that it is able to operationally retrieve ET in near-real time.

> You can check the TSEB code [here](https://github.com/hectornieto/pyTSEB/blob/9bb28da739090d36f9cc0c1a771efb897c05eb48/pyTSEB/TSEB.py#L479 "https://github.com/hectornieto/pyTSEB/blob/9bb28da739090d36f9cc0c1a771efb897c05eb48/pyTSEB/TSEB.py#L479")

We are going to estimate ET and validate it against the flux tower measurements. You can modify the relevant input canopy structural parameters and see whether they have or not a significant effect in flux retrievals. 

> You can always use the actual canopy structural parameters estimated for the field:
> 
> * Canopy height, $h_c \approx 4$ m.
>
> * Canopy cover at nadir, $f_c \approx 0.3$ m.
>
> * Canopy width to height ration,  $Shape \approx 1$ m.
>
> * Relative height of the bottom of the canopy, $h_{bottom} \approx 0.3$.
>
> * Relative position within the canopy with the maximum leaf density, $h_{max} \approx 0.5$.
>
> * Dominant leaf angle inclination, $LIDF \approx 57$ º.

Once you have pressed the `Estimate ET` buttom, a validation scatterplot will show-up. In addition, the flux outputs are saved in [./output/tseb_olive.csv](./output/tseb_olive.csv). Once you have run all the simulations and estimates, download this file as you will use it for the practical session on Water Use Efficiency.

In [ ]:
w_tseb = interactive(fn.run_tseb, {"manual": True, "manual_name": "Estimate ET"},
                     h_c=fn.w_hc, f_c=fn.w_fc, w_c=fn.w_wc, hb_ratio=fn.w_hb_ratio, 
                     h_c_max=fn.w_h_c_max, leaf_angle=fn.w_leaf_angle,
                     date_range=fn.w_dates)
display(w_tseb)

... and here you can see the output timeseries

In [ ]:
if not isinstance(w_tseb.result, type(None)):
    w_ts = interactive(fn.create_flux_timeseries,
                       date_range=fn.w_dates, 
                       df=fixed(w_tseb.result[0]), daily_df=fixed(w_tseb.result[1]), 
                       include_obs=fixed(True))
    display(w_ts)

# Conclusions
In this exercise we have seen that exchage of heat and vapour between ground, vegetation and the atmosphere depend on:
1. The topsoil moisture content, which drives the soil evaporation.
2. The root-zone soil moisture content, which drives the canopy transpiration and its photosynthetic capacity.
3. The canopy structure, which drives how efficiently the canopy dissipates the heat and vapour towards the atmosphere.

The radiometric land surface temperature is a key factor in order to evaluate the crop water stress and crop evapotranspiation. 

We have used and tested the TSEB model to estimate ET from radiometric LST, which is available in open source code [here](https://github.com/hectornieto/pyTSEB). In any case there exists a wide range of other energy balance models that allow esimating ET from remote sensing, all of them with their own strenghts and weakenesses. The final objective of this session is to introduce you the most relevant physical processes regarding ET, with the hope that you could form a critical knowledge allowing you to decide with model to use for your requirements. 